IMPORTACIÓN DE LIBRERÍAS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.decomposition import PCA

PROCESAMIENTO DE DATOS

In [2]:
data = pd.read_csv('C:/Users/JCfor/Desktop/Proyecto Tesis I/DatasetFinal.csv')
##Variable valor de traspaso
valor = data['Valor de traspaso'].str.split('-',expand = True)
valor.columns = ['MenorV', 'V_traspaso']
data = pd.concat( [ data,valor.V_traspaso ] , axis= 1)

##Variable Sueldo
data['Sueldo'] = data['Sueldo'].str.extract('(\d+)', expand=False).astype(float)
data['Sueldo'] = data['Sueldo']*1000
data.head()

data = data.drop( ['Inf', 'Source.Name', 'Rec', 'Nombre', 'Club', 'División', 'Posición', 'Edad', 'Sueldo', 'V_traspaso','Valor de traspaso' ], axis=1)

In [3]:
X = np.asanyarray(data.drop(columns=['Valoracion']))
Y=data['Valoracion']

print(X.shape)
print(Y.shape)

(3191, 47)
(3191,)


APLICANDO PCA

In [4]:
#ESCALARIZANDO LOS DATOS
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
pca = PCA(n_components=2)
pca.fit(X)
X = pca.transform(X)

CONSTRUCCIÓN DEL MODELO

In [6]:
def create_modelo():
    modelo = keras.Sequential([
        keras.layers.Dense(6, activation='sigmoid', input_shape=(2,)),
        keras.layers.Dense(1)
    ])

    sgd = tf.keras.optimizers.SGD(learning_rate=0.001,
    momentum=0.1)

    modelo.compile(optimizer = sgd,
                loss='mse',
                metrics=['mae'])
    return modelo

In [7]:
from keras.wrappers.scikit_learn import KerasRegressor
model = KerasRegressor(build_fn=create_modelo, epochs=600, batch_size=10)

C:\Users\JCfor\AppData\Local\Temp\ipykernel_22992\66332328.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_modelo, epochs=600, batch_size=10)


In [8]:
from sklearn.model_selection import ShuffleSplit,cross_val_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

shuffle_split=ShuffleSplit(test_size=0.2,train_size=0.6,n_splits=5)
scores=cross_val_score(model,X,Y,scoring='r2',cv=shuffle_split)

Epoch 1/600
192/192 [==============================] - 0s 799us/step - loss: 814.1711 - mae: 22.3241
Epoch 2/600
192/192 [==============================] - 0s 705us/step - loss: 108.8614 - mae: 8.3321
Epoch 3/600
192/192 [==============================] - 0s 782us/step - loss: 58.7308 - mae: 6.2200
Epoch 4/600
192/192 [==============================] - 0s 839us/step - loss: 42.1641 - mae: 5.1905
Epoch 5/600
192/192 [==============================] - 0s 763us/step - loss: 34.7964 - mae: 4.6656
Epoch 6/600
192/192 [==============================] - 0s 758us/step - loss: 31.3233 - mae: 4.3923
Epoch 7/600
192/192 [==============================] - 0s 882us/step - loss: 29.7088 - mae: 4.2743
Epoch 8/600
192/192 [==============================] - 0s 748us/step - loss: 29.0355 - mae: 4.2239
Epoch 9/600
192/192 [==============================] - 0s 740us/step - loss: 28.4574 - mae: 4.1875
Epoch 10/600
192/192 [==============================] - 0s 785us/step - loss: 28.2026 - mae: 4.1831
Epoch 

In [10]:
print("cross Validation scores:n {}".format(scores))
print("Average Cross Validation score :{}".format(scores.mean()))

cross Validation scores:n [0.92296966 0.9272613  0.91230821 0.93178721 0.93460188]
Average Cross Validation score :0.9257856540077279
